This notebook contains an example of using `redbiom` through it's Python API to extract a subset of American Gut Project samples. These data are then loaded into QIIME 2 for a mini beta-diversity analysis using UniFrac. This assumes we're using a QIIME 2 2018.11 environment that additionally has `redbiom` 0.3.0 installed. The exact commands I ran to install it are:

```
$ conda install nltk
$ pip install https://github.com/biocore/redbiom/archive/0.3.0.zip
```

In [1]:
import redbiom.summarize
import redbiom.search
import redbiom.fetch
import qiime2
import pandas as pd
pd.options.display.max_colwidth = 1000

The first thing we're going to do is gather the `redbiom` contexts. A context is roughly a set of consistent technical parameters. For example, the specific sequenced gene, the variable region within the gene, the length of the read, and how the operational taxonomic units were assessed.

The reason `redbiom` partitions data into contexts is because these technical details can lead to massive technical bias. The intention is to facilitate comparing "apples" to "apples". 

The context summarization returns a pandas `DataFrame` so it should be pretty friendly to manipulate.

In [2]:
contexts = redbiom.summarize.contexts()

In [3]:
contexts.shape

(104, 4)

In [7]:
contexts.head(2)

,ContextName,SamplesWithData,FeaturesWithData,Description
0,Pick_closed-reference_OTUs-Greengenes-ls454-16S-v6-150nt-bd7d4d,114,3115,Qiita context
1,Pick_closed-reference_OTUs-Greengenes-flx-16S-v4-100nt-a243a1,116,3306,Qiita context


At the present time, determining the context to use is a bit manual and requires some strung munging. Additional development is needed.

Let's take a look at the larger contexts.

In [8]:
contexts.sort_values('SamplesWithData', ascending=False).head()

,ContextName,SamplesWithData,FeaturesWithData,Description
49,Pick_closed-reference_OTUs-Greengenes-illumina-16S-v4-100nt-a243a1,129596,74983,Qiita context
65,Pick_closed-reference_OTUs-Greengenes-illumina-16S-v4-5c6506,128222,82492,Qiita context
39,Pick_closed-reference_OTUs-Greengenes-illumina-16S-v4-90nt-44feac,125354,73083,Qiita context
69,Deblur-NA-illumina-16S-v4-100nt-fbc5b2,123127,5587560,Qiita context
93,Deblur-NA-illumina-16S-v4-90nt-99d1d8,119538,4460311,Qiita context


For simplicity sake, let's take the first context. It's large, and the phylogeny associated with the operational taxonomic units is easy to get. We'll break down the meaning of the context name in a moment. In practice, you will _most likely_ want to use the Deblur data, however producing a reasonable tree from those data requies a slightly computationally expensive step, and I'm on my laptop right now with limited battery quite literally in the middle of nowhere on a bus in the Czech Republic.

## Using Deblur Context

In [9]:
ctx = contexts.sort_values('SamplesWithData', ascending=False).iloc[3]['ContextName']

In [10]:
ctx

'Deblur-NA-illumina-16S-v4-100nt-fbc5b2'

**for Daniel Notes below talks about Greengenes but we are using deblur as he recommends instaed**

Breaking this name into its constiuent pieces, this is a closed reference context meaning that operational taxonomic units were assessed against a reference database and sequences which did not recruit to the reference were discarded. The reference used is Greengenes, a common 16S reference database. The gene represented by the data is the 16S SSU rRNA gene, and specifically the V4 region of the gene. Finally, the fragments represented are truncated to 100 nucleotides. (Don't worry if this is all a lot of jargon. It is a lot of jargon. Please ask questions :)

So cool, we have a "context". What can we do now? Let's search for some sample identifiers based off of the metadata (i.e., variables) associated with the samples. Specifically, let's get some skin, oral and fecal samples. Be forewarned, the metadata search uses Python's `ast` module behind the scenes, so malformed queries at present produce tracebacks.

In [11]:
study_id = 10317  # the Qiita study ID of the American Gut Project is 10317

query = "where qiita_study_id==%d" % (study_id)
results = redbiom.search.metadata_full(query)

In [12]:
len(results)

21506

In [15]:
study_id = 10317  # the Qiita study ID of the American Gut Project is 10317
results = {}
for site in ['sebum', 'saliva', 'feces']:
    query = "where qiita_study_id==%d and env_material=='%s'" % (study_id, site)
    results[site] = redbiom.search.metadata_full(query)

In [16]:
for k, v in results.items():
    print(k, len(v))

feces 16207
saliva 1257
sebum 1136


## Want to get metadata for all samples and then export to csv

In [51]:
to_keep_all = []
for k, v in results.items():
    to_keep_all.extend(list(v))

In [52]:
to_keep_all
md_all, _ = redbiom.fetch.sample_metadata(to_keep_all, context=ctx)

In [53]:
md_all.shape

(14657, 52)

In [56]:
md_all.head(2)

,#SampleID,alcohol_types_beercider,alcohol_types_red_wine,alcohol_types_sour_beers,alcohol_types_spiritshard_alcohol,alcohol_types_unspecified,alcohol_types_white_wine,allergic_to_i_have_no_food_allergies_that_i_know_of,allergic_to_other,allergic_to_peanuts,...,subset_age,subset_antibiotic_history,subset_bmi,subset_diabetes,subset_healthy,subset_ibd,survey_id,taxon_id,title,weight_units
26674_10317.000001247,10317.000001247.26674,false,false,false,false,true,false,false,false,false,...,true,false,false,true,false,true,0fc2b33cc12ffa57,408170,American Gut Project,kilograms
26674_10317.000001820,10317.000001820.26674,true,false,false,true,false,false,true,false,false,...,true,true,true,true,true,true,744dfbc2b20daa5d,408170,American Gut Project,kilograms


**exporting metadata to .csv, (14657 rows and 52 columns)**

In [57]:
md_all.to_csv('AGP_metdata.csv')

The last output cell shows what these IDs look like. These are Qiita sample IDs.

Now that we have some samples, let's get some data! What we're going to do is ask `redbiom` to obtain the sample data, for our `to_keep` samples, from the context we previously selected. What's happening behind the scenes is that the API is pulling out sparse vectors corresponding to the number of individual sequences observed for each operational taxonomic unit per sample, and additionally unmunging the names (as `redbiom` normalizes sample and feature identifiers). The output is then aggregated into what's called a BIOM `Table`, which is really just a rich object wrapped around a `scipy.sparse` matrix. 

You may noice two outputs on the return. The one we're ignoring represents "ambiguous" samples. Some sample identifiers are associated with multiple sequenced samples. This is because some samples may "fail" sequencing, where they didn't yield sufficient sequence data, and were rerun. These "failures" are still represented in Qiita, but are differentiated by the actual sequencing run they were on. This doesn't matter for the moment though.

## Get the biom_table (OTU data) from deblur context

the following takes a while to run (!30 minutes)

In [58]:
biom_table, _ = redbiom.fetch.data_from_samples(ctx, to_keep_all)

In [74]:
biom_table

376923 x 14657 <class 'biom.table.Table'> with 2808216 nonzero entries (0% dense)

wow 376923 different rows (each row is a different OTU)

In [62]:
# lookup for ITU ID's and what they mean - linking them to higher level?

In [63]:
print(biom_table.head())

# Constructed from biom file
#OTU ID	10317.000066557.33038	10317.000046467.33048	10317.000084591.42561	10317.000029998.27270	10317.000009612.27865
TACGTATGGTGCGAGCGTTGTCCGGAATTATTGGGCGTAAAGGGTACGTAGGCGGTTAAACAAGTCAGATGTAAAAACTTGAGGCTCAACCTTATGTAGC	0.0	0.0	0.0	0.0	0.0
AACGTAGGTCACAAGCGTTGTCCGGAATTACTGGGTGTAAAGGGTGCGTAGACGGGAGAGCAAGTTAGTTGTGAAATCCCTCGGCTTAACTGAGGAACTG	0.0	0.0	0.0	0.0	0.0
TACGTATGGAGCAAGCGTTATCCGGATTTACTGGGTGTAAAGGGTGCGTAGGTGGCAGTGCAAGTCAGACGTGAAATCCATGGGCTCAACCCATGAACTG	0.0	0.0	0.0	0.0	0.0
TACGGAGGGGGCAAGCGTTGTTCGGATTTACTGGGCGTAAAGGGCGCGTAGGCGGCGCAACGAGTCACTTGTGAAATCTCCGGGCTTAACTCGGAACTGC	0.0	0.0	0.0	0.0	0.0
TACGTATGGTGCAAGCGTTATCCGGATTTACTGGATGTAAAGGGAGCGCAGGCGGTCTGGCAAGTCTGATGTGAAAATCCGGGGCTCAACTTCGGAACTG	0.0	0.0	0.0	0.0	0.0


**create a pandas sparse dataframe from the results and export to pickle file**

In [67]:
adf = pd.SparseDataFrame(biom_table.matrix_data)

In [69]:
adf.shape

(376923, 14657)

In [71]:
adf.to_pickle('biom_table.pkl')